# Regressão Neural Manual Simples
Este notebook demonstra a implementação de uma rede neural de regressão do zero, incluindo:

- Remoção de outliers via IQR
- Pré-processamento dos dados (normalização e split)
- Definição de funções de ativação e perda
- Classe de rede neural com uma camada oculta
- Treinamento e relatório de pesos efetivos
- Avaliação dos resultados e plotagem


## 0) Importações e Função de Remoção de Outliers
A função `remove_outliers_iqr` aplica a regra do intervalo interquartil (IQR) para detectar e remover valores extremos em qualquer coluna numérica.

In [ ]:
import pandas as pd
import numpy as np

def remove_outliers_iqr(df: pd.DataFrame, factor: float = 1.5, id_col: str = "ID") -> pd.DataFrame:
    num_cols = [c for c in df.select_dtypes(exclude="object").columns if c != id_col]
    mask = pd.Series(False, index=df.index)
    for col in num_cols:
        Q1, Q3 = df[col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower, upper = Q1 - factor * IQR, Q3 + factor * IQR
        mask |= (df[col] < lower) | (df[col] > upper)
    if mask.any():
        removed_ids = df.loc[mask, id_col].tolist()
        print(f"Outliers removidos (IDs): {removed_ids}")
    return df.loc[~mask].reset_index(drop=True)

## 1) Pré-processamento dos Dados
Carregamos o CSV, removemos outliers, separamos `X` e `y`, normalizamos com Min-Max e fizemos o split treino/teste.

In [ ]:
from sklearn.model_selection import train_test_split

def load_and_preprocess(filepath: str, factor: float = 1.5, test_size: float = 0.2, random_state: int = 42):
    df = pd.read_csv(filepath)
    df = remove_outliers_iqr(df, factor)
    features = [c for c in df.columns if c not in ("ID", "Price")]
    X_raw = df[features].values
    y_raw = df["Price"].values.reshape(-1, 1)
    X_min, X_max = X_raw.min(axis=0), X_raw.max(axis=0)
    X = (X_raw - X_min) / (X_max - X_min)
    y_min, y_max = y_raw.min(), y_raw.max()
    y = (y_raw - y_min) / (y_max - y_min)
    return train_test_split(X, y, test_size=test_size, random_state=random_state) + (features, y_min, y_max)

## 2) Definição da Rede Neural
Uma classe com:

- `forward`: cálculo das ativações e predição
- `backward`: atualização de pesos via retropropagação
- `train`: loop de treinamento com impressão de loss

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size: int, hidden_size: int, output_size: int, learning_rate: float):
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2 / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2 / hidden_size)
        self.b2 = np.zeros((1, output_size))
        self.lr = learning_rate

    def forward(self, X: np.ndarray) -> np.ndarray:
        self.Z1 = X.dot(self.W1) + self.b1
        self.A1 = np.maximum(0, self.Z1)
        return self.A1.dot(self.W2) + self.b2

    def backward(self, X: np.ndarray, y_true: np.ndarray, y_pred: np.ndarray) -> None:
        dZ2 = 2 * (y_pred - y_true) / y_true.size
        dW2 = self.A1.T.dot(dZ2)
        db2 = dZ2.sum(axis=0, keepdims=True)
        dA1 = dZ2.dot(self.W2.T)
        dZ1 = dA1 * (self.Z1 > 0)
        dW1 = X.T.dot(dZ1)
        db1 = dZ1.sum(axis=0, keepdims=True)
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X: np.ndarray, y: np.ndarray, epochs: int = 1000, print_every: int = 100) -> None:
        for epoch in range(1, epochs + 1):
            y_pred = self.forward(X)
            loss = np.mean((y - y_pred) ** 2)
            self.backward(X, y, y_pred)
            if epoch == 1 or epoch % print_every == 0:
                print(f"Epoch {epoch:>4d}, Loss = {loss:.6f}")

    def get_feature_weights(self) -> np.ndarray:
        return (self.W1.dot(self.W2)).flatten()

## 3) Treinamento e Relatório de Pesos
Executamos o treinamento e exibimos os pesos efetivos por atributo.

In [ ]:
# Execução principal
filepath = 'real_estate_dataset.csv'
X_train, X_test, y_train, y_test, features, y_min, y_max = load_and_preprocess(filepath)

nn = NeuralNetwork(input_size=X_train.shape[1], hidden_size=256, output_size=1, learning_rate=0.015)
print("Treinamento iniciado...")
nn.train(X_train, y_train, epochs=3000, print_every=200)
print("Treinamento concluído.")

In [ ]:
# Relatório de pesos
weights = nn.get_feature_weights()
print("\nPesos efetivos por atributo:")
for name, w in zip(features, weights):
    print(f"{name:<20s}: {w:+.6f}")

## 4) Avaliação e Plotagem
Calculamos MSE, R² e plotamos Real vs Predito.

In [ ]:
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

y_pred_norm = nn.forward(X_test)
y_pred = y_pred_norm * (y_max - y_min) + y_min
y_actual = y_test * (y_max - y_min) + y_min

mse = np.mean((y_actual - y_pred) ** 2)
r2 = r2_score(y_actual, y_pred)
print(f"MSE no teste (real): {mse:.2f}")
print(f"R² no teste: {r2:.4f}")

plt.figure(figsize=(6,6))
plt.scatter(y_actual, y_pred, alpha=0.6, edgecolor='k')
plt.plot([y_actual.min(), y_actual.max()], [y_actual.min(), y_actual.max()], 'r', linewidth=2)
plt.xlabel('Preço Real')
plt.ylabel('Preço Predito')
plt.title('Real vs Predito')
plt.tight_layout()
plt.show()

### Fim do notebook
Este notebook ilustra passo a passo a implementação manual de uma rede neural de regressão com remoção de outliers.